In [4]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

In [5]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward, contact_cost_weight, healthy_z_range, contact_force_range):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    contact_cost_weight = contact_cost_weight,
                    healthy_z_range=healthy_z_range,
                    contact_force_range=contact_force_range)
                   # render_mode='none')

In [6]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)           # Default circa 0.1; esploriamo da 0.05 a 0.2
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.5, 1.5)     # Default tipico è 1; esploriamo da 0.5 a 1.5
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.1, 1.0)               # Default tipico 0.5; esploriamo da 0.1 a 1.0
    healthy_reward = trial.suggest_float('healthy_reward', 0.5, 1.5)                   # Default tipico 1; esploriamo da 0.5 a 1.5
    
    # Parametri aggiuntivi per Ant-v5
    contact_cost_weight = trial.suggest_float('contact_cost_weight', 1e-4, 1e-3)  # Es. range intorno a 5e-4 come default
    healthy_z_lower = trial.suggest_float('healthy_z_lower', 0.1, 0.3)             # Per definire l'intervallo di altezze "sane"
    healthy_z_upper = trial.suggest_float('healthy_z_upper', 0.8, 1.2)
    contact_force_min = trial.suggest_float('contact_force_min', -1.0, -0.5)         # Modificabile se usi forze di contatto
    contact_force_max = trial.suggest_float('contact_force_max', 0.5, 1.0)  

    # Crea l'ambiente passando tutti i parametri
    env = make_env(
        reset_noise_scale,
        forward_reward_weight,
        ctrl_cost_weight,
        healthy_reward,
        contact_cost_weight=contact_cost_weight,
        healthy_z_range=(healthy_z_lower, healthy_z_upper),
        contact_force_range=(contact_force_min, contact_force_max)
    )
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello PPO
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])  
    # Per ambienti complessi come Ant, molti esperimenti usano gamma intorno a 0.99-0.995
    gamma = trial.suggest_float('gamma', 0.99, 0.999)
    gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)
    clip_range = trial.suggest_float('clip_range', 0.1, 0.3) 
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)
    
    # Nuovo iperparametro per la penalizzazione della varianza
    variance_penalty_weight = trial.suggest_float('variance_penalty_weight', 0.0, 0.5)

    # Crea ed allena il modello PPO
    model = PPO("MlpPolicy", env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                ent_coef=ent_coef,
                verbose=0)
    model.learn(total_timesteps=200000)

    # Valuta il modello su 200 episodi (200 è ottimale)
    episodes = 150
    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)

    # Calcola reward media e varianza
    mean_reward = np.mean(episode_rewards)
    reward_variance = np.var(episode_rewards)

    # Definisce l'obiettivo: massimizzare la reward media penalizzando la varianza
    score = mean_reward - variance_penalty_weight * reward_variance

    print(f'Mean is: {mean_reward}, Variance is: {reward_variance}\n')

    return score

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-11 14:33:10,145] A new study created in memory with name: no-name-62f66b6f-26af-4b89-bedc-d8d78f8f518b


/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_26250/1335275736.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-02-11 14:34:19,320] Trial 0 finished with value: -0.40184521675109863 and parameters: {'reset_noise_scale': 0.17328697116919345, 'forward_reward_weight': 0.8600760817025472, 'ctrl_cost_weight': 0.6533513452331322, 'healthy_reward': 0.8881430529953126, 'contact_cost_weight': 0.00021706031816801983, 'healthy_z_lower': 0.18487760276555745, 'healthy_z_upper': 0.8736510511245551, 'contact_force_min': -0.8504374331245832, 'contact_force_max': 0.7191725539574108, 'learning_rate': 0.00024366394019125905, 'n_steps': 4096, 'batch_size': 256, 'gamma': 0.9941622716086058, 'gae_lambda': 0.8603002500571459, 'clip_range': 0.274333840884758

Mean is: -0.3652510941028595, Variance is: 0.09386386722326279



[I 2025-02-11 14:35:35,460] Trial 1 finished with value: -2.3926029205322266 and parameters: {'reset_noise_scale': 0.15057781462445988, 'forward_reward_weight': 0.7623185185030785, 'ctrl_cost_weight': 0.5953641858803702, 'healthy_reward': 1.1364343732050866, 'contact_cost_weight': 0.00010699224876114234, 'healthy_z_lower': 0.2996324727057889, 'healthy_z_upper': 0.8878065627772774, 'contact_force_min': -0.7724886039906093, 'contact_force_max': 0.6613403758444628, 'learning_rate': 1.5661848374396858e-05, 'n_steps': 2048, 'batch_size': 128, 'gamma': 0.9932113330391995, 'gae_lambda': 0.9215302430788892, 'clip_range': 0.2399985875632972, 'ent_coef': 0.041977432377012694, 'variance_penalty_weight': 0.3700584298634591}. Best is trial 0 with value: -0.40184521675109863.


Mean is: -1.5405153036117554, Variance is: 2.3025755882263184



...

Mean is: -0.28820210695266724, Variance is: 10.735915184020996



[I 2025-02-11 14:53:10,665] Trial 14 finished with value: -2.359555721282959 and parameters: {'reset_noise_scale': 0.09211592686797554, 'forward_reward_weight': 1.1038949560115847, 'ctrl_cost_weight': 0.29231681368235013, 'healthy_reward': 1.0526105268110622, 'contact_cost_weight': 0.0005075844698602804, 'healthy_z_lower': 0.10588663663371299, 'healthy_z_upper': 1.004950338679169, 'contact_force_min': -0.7046212439205792, 'contact_force_max': 0.7860034751282772, 'learning_rate': 5.0060320604016884e-05, 'n_steps': 2048, 'batch_size': 64, 'gamma': 0.9984747555845591, 'gae_lambda': 0.8419580527663166, 'clip_range': 0.20480041559940193, 'ent_coef': 0.07625395348031483, 'variance_penalty_weight': 0.14255326547809696}. Best is trial 5 with value: 0.7956998348236084.


Mean is: -0.7026627659797668, Variance is: 11.622973442077637



[W 2025-02-11 14:54:01,567] Trial 15 failed with parameters: {'reset_noise_scale': 0.08141438982798507, 'forward_reward_weight': 1.2672251093423883, 'ctrl_cost_weight': 0.2904860914684766, 'healthy_reward': 0.5290929428345446, 'contact_cost_weight': 0.0007063858031248777, 'healthy_z_lower': 0.22485444184857956, 'healthy_z_upper': 1.0659064133405933, 'contact_force_min': -0.5933017073539808, 'contact_force_max': 0.9004982797402342, 'learning_rate': 0.0004601703506883517, 'n_steps': 2048, 'batch_size': 64, 'gamma': 0.9958292438961996, 'gae_lambda': 0.9995679087193103, 'clip_range': 0.203176645089613, 'ent_coef': 0.035897374727136125, 'variance_penalty_weight': 0.1021124680051293} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(tr

KeyboardInterrupt: 